In [49]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

import pandas as pd

import math

## 2) Teniendo en cuenta las reviews que reciben las aplicaciones, devolver una (al azar) de la aplicación que haya recibido la mayor cantidad de reviews positivas. (⭐)

In [50]:
# Crear una instancia de SparkSession y SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Leer los archivos CSV y crear RDDs
df_apps = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("./dataset/googleplaystore.csv", header=True, inferSchema=True)
rdd_apps = df_apps.rdd

df_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("./dataset/googleplaystore_user_reviews.csv", header=True, inferSchema=True)
rdd_reviews = df_reviews.rdd

# Filtrar solo reseñas positivas (asumiremos que Sentiment es "Positive" en el segundo conjunto de datos)
positive_reviews_rdd = rdd_reviews.filter(lambda x: x['Sentiment'] == 'Positive')

# Filtrar solo valores numéricos en la columna "Reviews"
rdd_apps = rdd_apps.filter(lambda x: x['Reviews'].isnumeric())

# Convertir la columna "Reviews" a un tipo numérico
rdd_apps = rdd_apps.map(lambda x: (x['App'], int(x['Reviews'])))

# Obtener la aplicación con la mayor cantidad de reseñas positivas y su número de reseñas
app_with_most_positive_reviews = rdd_apps.reduce(lambda a, b: a if a[1] >= b[1] else b)

# Filtrar todas las reseñas de la aplicación con la mayor cantidad de reseñas positivas
reviews_of_app_with_most_positive_reviews = positive_reviews_rdd.filter(lambda x: x['App'] == app_with_most_positive_reviews[0])

# Seleccionar una reseña al azar de esa aplicación sin usar collect()
random_review = reviews_of_app_with_most_positive_reviews.takeSample(False, 1)

# Imprimir la aplicación y la reseña seleccionada al azar
print("Aplicación con más reseñas positivas:", app_with_most_positive_reviews[0])
print("Reseña seleccionada al azar:")
print("Translated_Review:", random_review[0]['Translated_Review'])
print("Sentiment:", random_review[0]['Sentiment'])

Aplicación con más reseñas positivas: Facebook
Reseña seleccionada al azar:
Translated_Review: I believe lot Google need slow little. There enough recent improvements satisfy social media world time being. Let us digest latest exponential leaps brilliant design development user friendly fixes wiring. I mean, EVERY WEEK !
Sentiment: Positive


## 31) ¿Cuál es la app con mayor cantidad de instalaciones? (⭐)

In [66]:
# ¿Cuál es la app con mayor cantidad de instalaciones?

# Crear una instancia de SparkSession y SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Leer los archivos CSV y crear RDDs
df_apps = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("./dataset/googleplaystore.csv", header=True, inferSchema=True)
rdd_apps = df_apps.rdd

# Eliminar la fila que esta mal cargada
rdd_apps = rdd_apps.filter(lambda x: x['Installs'] != 'Free')

# Convertir la columna "Installs" a un tipo numérico
rdd_apps = rdd_apps.map(lambda x: (x['App'], int(x['Installs'].replace('+', '').replace(',', ''))))

# Obtener la aplicación con la mayor cantidad de instalaciones y su número de instalaciones
app_with_most_installs = rdd_apps.reduce(lambda a, b: a if a[1] >= b[1] else b)

# Imprimir la aplicación con la mayor cantidad de instalaciones
print("Aplicación con más instalaciones:", app_with_most_installs[0])


Aplicación con más instalaciones: Google Play Books


In [76]:
from pyspark.sql import SparkSession

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName("AppInstalls").getOrCreate()

# Leer el archivo CSV directamente como un RDD
rdd_apps = spark.sparkContext.textFile("./dataset/googleplaystore.csv")

# Filtrar y dividir las líneas correctamente
header = rdd_apps.first()
rdd_apps = rdd_apps.filter(lambda line: line != header).map(lambda line: line.split(','))

# Función para manejar la conversión a número con manejo de errores
def safe_int(value):
    try:
        return int(value)
    except ValueError:
        return 0

# Obtener la aplicación con la mayor cantidad de instalaciones
app_with_most_installs = rdd_apps.map(lambda x: (x[0], safe_int(x[5].replace('+', '').replace(',', '')))).max(key=lambda x: x[1])

# Imprimir la aplicación con la mayor cantidad de instalaciones
print("Aplicación con más instalaciones:", app_with_most_installs[0])

23/10/05 11:13:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Aplicación con más instalaciones: "360 Security - Free Antivirus


In [83]:
# ejemplo de uso de safe_int
a = 'Varies'
b = a.replace('+', '').replace(',', '')

print(safe_int('1,000,000+'))
print(safe_int('1,000,000'))
print(safe_int('Varies with device'))

print(safe_int(b))
c = safe_int(b)

0
0
0
0


## 10) Cual es la categoría con mayor promedio de apps que hayan sido al menos 1 vez calificadas como positivas (⭐⭐)

In [67]:
# Cual es la categoría con mayor promedio de apps que hayan sido al menos 1 vez calificadas como positivas 

# Crear una instancia de SparkSession y SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Leer los archivos CSV y crear RDDs
df_apps = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("./dataset/googleplaystore.csv", header=True, inferSchema=True)
rdd_apps = df_apps.rdd

df_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("./dataset/googleplaystore_user_reviews.csv", header=True, inferSchema=True)
rdd_reviews = df_reviews.rdd

# Filtrar solo reseñas positivas (asumiremos que Sentiment es "Positive" en el segundo conjunto de datos)
positive_reviews_rdd = rdd_reviews.filter(lambda x: x['Sentiment'] == 'Positive')








